### Correction of Topological Errors in Traces 
Multiple stacks of neuron images were traced by two users to create a labeled dataset and assess the baseline of inter-user variability. All traces were then disconnected at branch points and a deep neural network was trained to detect the correct way of reconnecting the branches. Custom morphological features were generated for each cluster of branch points, in a way that is dependent on a merging scenario but invariant to translations, rotations, and reflections of the cluster in the imaging plane. The features and image volume centered at the branch point were used for training a neural network that concatenates these input streams and outputs the confidence measure for different branch merging scenarios.

In [31]:
# Import 
import numpy as np
from numpy.random import seed
from keras.layers import Dense, Activation, BatchNormalization, Concatenate
from keras.callbacks import TensorBoard
import datetime
import scipy.io as sio

In [32]:
ImagetoTest = 1
run = 0
UseIMage = True

In [33]:
from variables import *
from models import *
from utils import *
from datas import *
import AT_Classes as Classes

In [34]:
lst_XYZ = ['False']
lst_useImage = ['True']
useEndpointFeatures = 'True'
kernel_initializer='he_uniform'
rotation_degrees = [0,90,180,270]
flips = ['right']
UseConv = True
rotated_IMs = np.zeros([13,13,13,len(rotation_degrees)])

In [35]:
epoch  = 100
batch_size = 50 #50
verbose=1 #verbose=1 will show you an animated progress bar
doSMOTE = False #do replicate data using SMOTE method
learning_Rate = 0.001 # default  =0.01

In [36]:
x = datetime.datetime.today()
nowTimeDate = x.strftime("%b_%d_%H_%M")
# PltNAme = 'AT_XYZ_is_'+str(useXYZ_Positions)+'_'+str(ImagetoTest)+'_run=2'+nowTimeDate

PltNAme = 'NEW_SmallUnet1TEST18INV_FEATURES_CONV=' + str(UseConv) + '_LR=' + str(learning_Rate) + '_100_sce_' + str(
    kernel_initializer) + '_IM=' + str(ImagetoTest) + 'bchSiz=' + str(batch_size) + '_Use_IM=' + str(
    UseIMage) + '_Epoch=' + str(
    epoch) + '_run=' + str(run + 1)
print(PltNAme)

NEW_SmallUnet1TEST18INV_FEATURES_CONV=True_LR=0.001_100_sce_he_uniform_IM=1bchSiz=50_Use_IM=True_Epoch=100_run=1


In [37]:
# filepath = 'E:\AutomatedTraceResults\DataForConnectingTraining\Data_For_AE_BranchScenarios\IMonce_limit100scen_NEW_Inv_FEATURES.mat'
filepath = root_dir+'/data/datafeed/IMonce_limit100scen_NEW_Inv_FEATURES.mat'
print(filepath)
ScenariosData = sio.loadmat(filepath)

C:/Users/Seyed/Documents/TraceProofreading/data/datafeed/IMonce_limit100scen_NEW_Inv_FEATURES.mat


In [38]:
IMnums = ScenariosData['IMnum']
Features = ScenariosData['NewFeatures']
IMs = ScenariosData['IMs']
Scenarios = ScenariosData['Scenarios']
Labels = ScenariosData['Labels']

In [39]:
maxNumPoints = 12
# IMsTrain = []
# FeatureTrain = []
# LabelsTrain = []
# ScenariosTrain = []

# IMsTest = []
# FeatureTest = []
# LabelsTest = []
# ScenariosTest = []

UseUpper = False
numScenarios = Scenarios.shape
counter = 0

In [42]:
for i in range(numScenarios[1]):
    scenario = Scenarios[0,i]

    IM = IMs[0,i]
    # print(IM.shape)
    Feature = Features[0, i]


    # if scenario.shape[0] == 3:
    #     maxNumPoints = 3

    # for r in range(len(rotation_degrees)):
    #     degree = rotation_degrees[r]
    #     print(degree)
    #     rotated_IMs[:,:,:,r] = scipy.ndimage.interpolation.rotate(IM, degree, mode='nearest', reshape=False)
    #     # IM_Proj = Classes.IM3D.Z_Projection(rotated_IMs[:,:,:,r])
    #     # Classes.IM3D.plt(IM_Proj)

    Label = Labels[0, i]
    # print(scenarios.shape)
    # if scenario.any():
    # scenarios.shape[2]

    S = Classes.cl_scenario(maxNumPoints, scenario.shape[0],scenario,0)
    if UseUpper:
        scenario_arr = S.getUpperArr()
    else:
        scenario_arr = S.getWholeArr()

    if IMnums[0, i] != ImagetoTest:
        ScenariosTrain.append(scenario_arr)
        IMsTrain.append(IM)

        FeatureTrain.append(Feature)

        LabelsTrain.append(Label)

    else:
        ScenariosTest.append(scenario_arr)
        IMsTest.append(IM)
        FeatureTest.append(Feature)

        LabelsTest.append(Label)

NameError: name 'FeatureTest' is not defined

In [ ]:
ScenariosTrain = np.asarray(ScenariosTrain, dtype=np.float)
IMsTrain = np.asarray(IMsTrain, dtype=np.float)
IMsTrain3D = IMsTrain
FeatureTrain = np.asarray(FeatureTrain, dtype=np.float)
FeatureTrain = FeatureTrain[:,0,:]


LabelsTrain = np.asarray(LabelsTrain, dtype=np.float)
LabelsTrain = LabelsTrain[:,0]
LabelsTrain = LabelsTrain[:,0]
IMsTrain1 = np.reshape(IMsTrain, [IMsTrain.shape[0],np.product(IMsTrain[0,:,:,:].shape)])

ScenariosTest = np.asarray(ScenariosTest, dtype=np.float)
IMsTest = np.asarray(IMsTest, dtype=np.float)
IMsTest3D = IMsTest
FeatureTest = np.asarray(FeatureTest, dtype=np.float)
FeatureTest = FeatureTest[:,0,:]

# Endpoint_features_Test = Endpoint_features_Test[:, :, 0]
LabelsTest = np.asarray(LabelsTest, dtype=np.float)
LabelsTest = LabelsTest[:,0]
LabelsTest = LabelsTest[:,0]
IMsTest1 = np.reshape(IMsTest, [IMsTest.shape[0],np.product(IMsTest[0,:,:,:].shape)])

# Sbhuffle Data
indices = np.arange(len(ScenariosTrain))
np.random.shuffle(indices)
ScenariosTrain = ScenariosTrain[indices]
IMsTrain = IMsTrain[indices]
FeatureTrain = FeatureTrain[indices]

LabelsTrain = LabelsTrain[indices]

XIMs_train = IMsTrain1

XFeature_train = FeatureTrain

XScenarios_train = ScenariosTrain
yIMs_train = LabelsTrain

XIMs_test = IMsTest1
XFeature_test = FeatureTest

XScenarios_test = ScenariosTest
yIMs_test = LabelsTest
yFeature_test = LabelsTest
yScenarios_test = LabelsTest

print(XIMs_train.shape)
print(XFeature_train.shape)
print(XScenarios_train.shape)
print(yIMs_train.shape)

In [ ]:
# to ignore image data
if UseIMage == False:
    XIMs_train = np.zeros(XIMs_train.shape)
    XIMs_test = np.zeros(XIMs_test.shape)
    print('Not Using Image')

In [ ]:
import keras
from keras.layers import Conv3D, MaxPooling3D, Input, Dense, Flatten
from keras.layers import (Concatenate, Conv3D, Dropout, Input,
                          Dense, MaxPooling3D, UpSampling3D, Activation, Reshape, Lambda,
                          Permute)

In [ ]:
def ConvolutionBlock(x, name, fms, params):
    x = Conv3D(filters=fms, **params, name=name+"_conv0")(x)
    x = BatchNormalization(name=name+"_bn0")(x)
    x = Activation("relu", name=name+"_relu0")(x)

    x = Conv3D(filters=fms, **params, name=name+"_conv1")(x)
    x = BatchNormalization(name=name+"_bn1")(x)
    x = Activation("relu", name=name)(x)
    return x

if UseConv:
    fms = 8
    input1 = Input(shape=(13, 13, 13, 1), name="inputs")

    params = dict(kernel_size=(3, 3, 3), activation=None,
                  padding="same", kernel_initializer="he_uniform")

    # Transposed convolution parameters
    params_trans = dict(kernel_size=(2, 2, 2), strides=(2, 2, 2), padding="same")

    # BEGIN - Encoding path
    encodeA = ConvolutionBlock(input1, "encodeA", fms, params)
    poolA = MaxPooling3D(name="poolA", pool_size=(2, 2, 2))(encodeA)

    encodeB = ConvolutionBlock(poolA, "encodeB", fms * 2, params)
    poolB = MaxPooling3D(name="poolB", pool_size=(2, 2, 2))(encodeB)
    #
    # encodeC = ConvolutionBlock(poolB, "encodeC", fms * 4, params)
    # poolC = MaxPooling3D(name="poolC", pool_size=(2, 2, 2))(encodeC)
    #
    # encodeD = ConvolutionBlock(poolC, "encodeD", fms * 8, params)

    x2 = Flatten()(poolB)
else:
    # # Relu
    input1 = keras.layers.Input(shape=(XIMs_train.shape[1],))
    # ,kernel_regularizer=keras.regularizers.l2(l=0.2)
    x1 = keras.layers.Dense(32, input_dim=XIMs_train.shape[1], activation='relu')(input1)
    x2 = keras.layers.Dense(16, input_dim=XIMs_train.shape[1], activation='relu')(x1)

In [ ]:
# Relu
input3 = keras.layers.Input(shape=(XFeature_train.shape[1],))
xxx0 = keras.layers.Dense(32, input_dim=XFeature_train.shape[1], activation='relu')(input3)
xxx1 = keras.layers.Dense(16, input_dim=XFeature_train.shape[1], activation='relu')(xxx0)
# xxx2 = keras.layers.Dense(8, input_dim=XFeature_train.shape[1], activation='relu')(xxx1)
# xxx2 = keras.layers.Dense(16, activation='relu')(xxx1)
# xxx3 = keras.layers.Dense(8, activation='relu')(xxx2)

In [ ]:
combined = keras.layers.concatenate([x2, xxx1])
# Relu
out = keras.layers.Dense(4, activation='sigmoid')(combined)

In [ ]:
out1 = keras.layers.Dense(1, activation='sigmoid')(out)
model = keras.models.Model(inputs=[input1, input3], outputs=out1)

In [ ]:
optimizer = keras.optimizers.Adam(lr=learning_Rate)

In [43]:
model.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'],

                          )

NameError: name 'model' is not defined

In [ ]:
tensorboard = TensorBoard(log_dir="E:/AutomatedTracing/AutomatedTracing/Python/logs/"+PltNAme)
# tensorboard --logdir=E:\AutomatedTracing\AutomatedTracing\Python\logs
# http://localhost:6006/#scalars&run=AT_All3&runSelectionState=...%3D%3D&_smoothingWeight=0.586

# checkpoint
from keras.callbacks import ModelCheckpoint
filepath= root_dir+'/data/models/'+PltNAme+"_weights.max_val_acc.hdf5"
checkpoint_max_val_acc = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

filepath = root_dir+'/data/models/'+ PltNAme + "_weights.min_val_acc.hdf5"
checkpoint_min_val_acc = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
                                         mode='min')

filepath = root_dir+'/data/models/'+ PltNAme + "_weights.max_val_loss.hdf5"
checkpoint_max_val_loss = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

filepath = root_dir+'/data/models/'+ PltNAme + "_weights.min_val_loss.hdf5"
checkpoint_min_val_loss = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,
                                          mode='min')

In [ ]:
if UseConv:
    X_IMs = IMsTrain3D.reshape(IMsTrain3D.shape[0], IMsTrain3D.shape[1], IMsTrain3D.shape[2], IMsTrain3D.shape[3],
                       1)
else:
    X_IMs = XIMs_train

In [ ]:
print(X_IMs.shape)

In [ ]:
history = model.fit([X_IMs,XFeature_train],
                                yIMs_train,
                                epochs=100,
                                batch_size=batch_size,
                                validation_split=0.33,
                                verbose=1,
                                callbacks=[checkpoint_max_val_acc,checkpoint_min_val_acc,checkpoint_max_val_loss,checkpoint_min_val_loss,tensorboard])

In [ ]:
# model.save(root_dir+'/data/models/'+PltNAme+'.h5')
# print(model.summary())
# from keras.utils.vis_utils import plot_model
# logpath = root_dir+'/data/models/'+PltNAme+'.png'
# print(logpath)
# plot_model(model, to_file=logpath, show_shapes=True, show_layer_names=True)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot( np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();